In [ ]:
# chatgpt code for gibbs sampling

import numpy as np

def gibbs_sampling_hmm(observed_data, num_states, num_samples, burn_in):
    # Initialize model parameters randomly
    transition_probs = np.random.rand(num_states, num_states)
    transition_probs /= np.sum(transition_probs, axis=1, keepdims=True)
    emission_means = np.random.rand(num_states, observed_data.shape[1])
    emission_variances = np.random.rand(num_states)
    hidden_states = np.zeros(len(observed_data), dtype=int)
    
    # Run Gibbs sampler
    for i in range(num_samples + burn_in):
        # Sample hidden states given observed data and model parameters
        for t in range(len(observed_data)):
            if t == 0:
                hidden_states[t] = np.random.randint(num_states)
            else:
                probs = transition_probs[hidden_states[t-1], :]
                hidden_states[t] = np.random.choice(num_states, p=probs)
        
        # Update transition probabilities given hidden states
        for s in range(num_states):
            for s_next in range(num_states):
                count = 0
                for t in range(1, len(observed_data)):
                    if hidden_states[t-1] == s and hidden_states[t] == s_next:
                        count += 1
                transition_probs[s, s_next] = np.random.beta(1 + count, 1 + np.sum(hidden_states[:-1] == s) - count)
        
        # Update emission parameters given hidden states and observed data
        for s in range(num_states):
            samples = observed_data[hidden_states == s, :]
            mean = np.mean(samples, axis=0)
            covariance = np.cov(samples.T)
            emission_means[s] = np.random.multivariate_normal(mean, covariance / len(samples))
            emission_variances[s] = np.random.gamma(len(samples) * observed_data.shape[1] / 2, 2 / np.sum((samples - mean)**2))
        
        # Store model parameters after burn-in period
        if i >= burn_in:
            yield (transition_probs, emission_means, emission_variances)


In [ ]:
import numpy as np

def gibbs_sampling_arhmm(observed_data, num_states, lag_order, num_samples, burn_in):
    # Initialize model parameters randomly
    transition_probs = np.random.rand(num_states, num_states)
    transition_probs /= np.sum(transition_probs, axis=1, keepdims=True)
    emission_weights = np.random.rand(observed_data.shape[1], num_states)
    emission_variances = np.random.rand(observed_data.shape[1], num_states)
    lag_weights = np.random.rand(observed_data.shape[1], num_states, lag_order)
    hidden_states = np.zeros(len(observed_data), dtype=int)
    
    # Pad observed data with zeros to allow for lagged observations
    padded_data = np.vstack([np.zeros((lag_order, observed_data.shape[1])), observed_data])
    
    # Run Gibbs sampler
    for i in range(num_samples + burn_in):
        # Sample hidden states given observed data and model parameters
        for t in range(len(observed_data)):
            if t == 0:
                hidden_states[t] = np.random.randint(num_states)
            else:
                probs = transition_probs[hidden_states[t-1], :]
                hidden_states[t] = np.random.choice(num_states, p=probs)
        
        # Update transition probabilities given hidden states
        for s in range(num_states):
            for s_next in range(num_states):
                count = 0
                for t in range(1, len(observed_data)):
                    if hidden_states[t-1] == s and hidden_states[t] == s_next:
                        count += 1
                transition_probs[s, s_next] = np.random.beta(1 + count, 1 + np.sum(hidden_states[:-1] == s) - count)
        
        # Update emission weights, variances, and lag weights given hidden states and observed data
        for s in range(num_states):
            samples = observed_data[hidden_states == s, :]
            for f in range(observed_data.shape[1]):
                mean = np.mean(samples[:, f])
                variance = np.var(samples[:, f])
                emission_weights[f, s] = np.random.normal(mean, np.sqrt(variance / len(samples)))
                emission_variances[f, s] = np.random.gamma(len(samples) / 2, 2 / np.sum((samples[:, f] - mean)**2))
                for l in range(lag_order):
                    lagged_samples = padded_data[l:len(observed_data)+l, :]
                    lag_mean = np.mean(lagged_samples[hidden_states == s, f])
                    lag_var = np.var(lagged_samples[hidden_states == s, f])
                    lag_weights[f, s, l] = np.random.normal(lag_mean, np.sqrt(lag_var / len(lagged_samples)))
        
        # Store model parameters after burn-in period
        if i >= burn_in:
            yield (transition_probs, emission_weights, emission_variances, lag_weights)
